In [1]:
import os
import re
import time
import urllib
import requests
from django.core.files import File
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
import json

In [2]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')

In [4]:
driver.get('https://www.dabangapp.com/room/5e992d31d9d3a15c88491696')

In [5]:
post_type = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/p/span')
post_type = post_type.get_attribute('innerText')

# 방 정보 설명
description = driver.find_elements_by_xpath("/html/body/div[1]/div/div[4]/div/div")
description = description[0].get_attribute("innerText")
description.replace("\n", "")

# 매물 형식
unrefined_salesform = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/div')
salesForm = unrefined_salesform[0].get_attribute("innerText")
salesForm = salesForm.replace('/', ' ')
salesForm = salesForm.replace('\n', '')
salesForm = salesForm.split()
salesType = salesForm[0]  # sales type


In [6]:
print(post_type, salesType)

아파트 월세


In [7]:
complex_detail_url = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/a')
complex_detail_url = complex_detail_url.get_attribute('href')
complex_detail_url

'https://www.dabangapp.com/complex/58291f8aa0c08473a04bb32c841c'

In [12]:
driver.get(complex_detail_url)

In [13]:
apart_name = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div/h1')
apart_name = apart_name.get_attribute('innerText')
apart_name

'트리마제'

In [14]:
made = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/p[2]')
made = made.get_attribute('innerText')
made

'2017.05'

In [15]:
total_citizen = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[2]/p[2]')
total_citizen = total_citizen.get_attribute('innerText')
total_citizen

'688세대'

In [16]:
personal_park = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[3]/p[2]')
personal_park = personal_park.get_attribute('innerText')
if ' ' in personal_park:
    personal_park = personal_park.split(' ')
    personal_park = personal_park[1]
personal_park

'1.6대'

In [17]:
# 총 동 수 
total_number = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[4]/p[2]')
total_number = total_number.get_attribute('innerText')
total_number

'4동'

In [18]:
heating_system = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[5]/p[2]')
heating_system = heating_system.get_attribute('innerText')
heating_system

'개별난방'

In [19]:
min_max_floor = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[6]/p[2]')
min_max_floor = min_max_floor.get_attribute('innerText')
min_max_floor

'28-47층'

In [20]:
buildingType = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/table/tbody/tr[1]/td[1]')
buildingType = buildingType.get_attribute('innerText')
buildingType

'아파트'

In [21]:
constructionCompany = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/table/tbody/tr[1]/td[2]')
constructionCompany = constructionCompany.get_attribute('innerText')
constructionCompany

'두산중공업(주)'

In [22]:
fuel = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/table/tbody/tr[1]/td[4]')
fuel = fuel.get_attribute('innerText')
fuel

'도시가스'

In [23]:
complex_type = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/table/tbody/tr[2]/td[1]')
complex_type = complex_type.get_attribute('innerText')
complex_type

'계단식'

In [24]:
# 용적률
floorAreaRatio = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/table/tbody/tr[2]/td[2]')
floorAreaRatio = floorAreaRatio.get_attribute('innerText')
floorAreaRatio

'319%'

In [25]:
# 건폐율
dryWasteRate = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/table/tbody/tr[2]/td[3]')
dryWasteRate = dryWasteRate.get_attribute('innerText')
dryWasteRate

'16%'

In [26]:
# 단지평당가 매매 
complexSale = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[1]/div/div/div/div[1]/p[3]')
complexSale = complexSale.get_attribute('innerText')
complexSale 

'-'

In [27]:
# 단지평당가 전세
complexPrice = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[1]/div/div/div/div[1]/p[5]')
complexPrice = complexPrice.get_attribute('innerText')
complexPrice

'-'

In [28]:
areaSale = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[1]/div/div/div/div[2]/p[3]')
areaSale = areaSale.get_attribute('innerText')
areaSale

'2745만/3.3m²당'

In [29]:
areaPrice = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[1]/div/div/div/div[2]/p[5]')
areaPrice = areaPrice.get_attribute('innerText')
areaPrice

'1602만/3.3m²당'

In [107]:
div_list = driver.find_elements_by_xpath('/html/body/div[1]/div/div[3]/div/div/div')

recommend_image_list = []

for i, url in enumerate(div_list):
    try:
        cls_name = url.get_attribute('class')
        cls_name = cls_name.split(' ')
        cls_name = cls_name[1]
        photo = driver.execute_script(
            f'return window.getComputedStyle(document.querySelector(".{cls_name}"),":after").getPropertyValue("background")')
        recommend_image = re.findall(r'"(.*?)"', photo)
        recommend_image_list.append(recommend_image[0])
    except IndexError:
        pass

In [108]:
recommend_image_list

['https://d2o59jgeq8ig2.cloudfront.net/complex/1024/4baadc0d-64cd-4db0-9e1d-99ef82a1146d',
 'https://d2o59jgeq8ig2.cloudfront.net/complex/1024/e64d5ff6-e831-42b3-9289-6a31a6a37c5a',
 'https://d2o59jgeq8ig2.cloudfront.net/complex/1024/ea036552-354b-4c08-85bd-d364e2400cf9',
 'https://d2o59jgeq8ig2.cloudfront.net/complex/1024/2d74ed1d-0e14-4c28-88d3-79e8e4113cc7',
 'https://d2o59jgeq8ig2.cloudfront.net/complex/1024/f05fe592-e5dd-4cb2-842b-7713c7a85103']

# 추천 단지

In [105]:
# 아파트 단지 이미지 div
div_list = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[8]/div/ul/li/div/a/div')

In [44]:
# 추천 단지 아파트 이름
recommend_apat_name_list = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[8]/div/ul/li/div/a/p[1]')

In [48]:
# 추천 단지 아파트
recommend_apat_type_list = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[8]/div/ul/li/div/a/p[2]/span[1]')

In [52]:
# 추천 단지 총 세대 수
recommend_apat_total_citizen_list = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[8]/div/ul/li/div/a/p[2]/span[2]')

In [54]:
# 추천 단지 설립일자 리스트
recommend_apat_build_date_list = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[8]/div/ul/li/div/a/p[2]/span[3]')

In [58]:
# 추천 단지 주소 리스트
recommend_apat_address_list = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[8]/div/ul/li/div/a/p[3]')

In [61]:
# 추천 단지 정보 링크 리스트
recommend_apat_link_list = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[8]/div/ul/li/ul/li/a')

In [106]:

for i, url in enumerate(div_list):
    cls_name = url.get_attribute('class')
    cls_name = cls_name.split(' ')
    cls_name = cls_name[1]

    photo = driver.execute_script(
                f'return window.getComputedStyle(document.querySelector(".{cls_name}"),":before").getPropertyValue("background")')
    recommend_image_url = re.findall(r'"(.*?)"', photo)
    print('추천단지 이미지', recommend_image_url[0])
    recommend_apat_name = recommend_apat_name_list[i].get_attribute('innerText')
    print('추천단지 아파트 이름', recommend_apat_name)
    recommend_apat_type = recommend_apat_type_list[i].get_attribute('innerText')
    print('추천단지 아파트 타입', recommend_apat_type)
    recommend_apat_total_citizen = recommend_apat_total_citizen_list[i].get_attribute('innerText')
    print('추천 단지 총 세대 수', recommend_apat_total_citizen)
    recommend_apat_build_date = recommend_apat_build_date_list[i].get_attribute('innerText')
    print('추천 단지 설립 일자', recommend_apat_build_date)
    recommend_apat_address = recommend_apat_address_list[i].get_attribute('innerText')
    print('추천 단지 주소', recommend_apat_address)
    recommend_apat_link = recommend_apat_link_list[i].get_attribute('href')
    print('추천 단지 해당 링크', recommend_apat_link)
    
    print('\n')


추천단지 이미지 https://d2o59jgeq8ig2.cloudfront.net/complex/1024/48c425e7-7ffd-497b-826d-848f09445b54
추천단지 아파트 이름 신동아
추천단지 아파트 타입 아파트
추천 단지 총 세대 수 636세대
추천 단지 설립 일자 1995.05
추천 단지 주소 서울특별시 성동구 행당동
추천 단지 해당 링크 https://www.dabangapp.com/complex/58291f83f59c8673a0fa98cd41c0


추천단지 이미지 https://d2o59jgeq8ig2.cloudfront.net/complex/1024/e43fdadc-132f-4c15-955e-cbed97ec17a6
추천단지 아파트 이름 힐스테이트서울숲리버
추천단지 아파트 타입 아파트
추천 단지 총 세대 수 606세대
추천 단지 설립 일자 2018.02
추천 단지 주소 서울특별시 성동구 금호동4가
추천 단지 해당 링크 https://www.dabangapp.com/complex/58291f8a0f97eb73a071e214d956


추천단지 이미지 https://d2o59jgeq8ig2.cloudfront.net/complex/1024/8722fdd0-5a65-41ec-995c-4bc29f2c54be
추천단지 아파트 이름 서울숲더샵
추천단지 아파트 타입 아파트
추천 단지 총 세대 수 495세대
추천 단지 설립 일자 2014.09
추천 단지 주소 서울특별시 성동구 행당동
추천 단지 해당 링크 https://www.dabangapp.com/complex/58291f889dd50c73a059ee805472


추천단지 이미지 https://d2o59jgeq8ig2.cloudfront.net/complex/1024/558bff38-121b-48dc-a445-4fa770770c2e
추천단지 아파트 이름 행당삼부
추천단지 아파트 타입 아파트
추천 단지 총 세대 수 498세대
추천 단지 설립 일자 1998.07
추천 단지 주소 서울특별시 성동구 행